# Images dataset Preparing for training.
## (Images Dataset Transform and Loader)

In [ ]:
import torchvision
import torch
import torchvision.transforms as transforms
import os
import matplotlib.pyplot as plt
import numpy as np
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

## Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Check location of dataset

In [ ]:
os.listdir('./drive/MyDrive/training')

['training']

In [ ]:
training_dataset_path = "./drive/MyDrive/training/training"

In [ ]:
testing_dataset_path = './drive/MyDrive/validation/validation'

## Transforms

In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(p = 0.25),
    transforms.RandomRotation(10), #10 degree rotation
    transforms.ToTensor(),# necessary for pytorch
    #transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
])

In [ ]:
test_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

## Creat Train and Test dataset with applied transforms

In [ ]:
Train_Ds =torchvision.datasets.ImageFolder(root = training_dataset_path, transform = train_transforms)
Test_Ds =torchvision.datasets.ImageFolder(root = testing_dataset_path, transform = test_transforms)

In [ ]:
print(type(Train_Ds))

<class 'torchvision.datasets.folder.ImageFolder'>


## Creat Loader

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset = Train_Ds, batch_size = 32, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = Test_Ds, batch_size = 32, shuffle = False)


## Show Transformed Images

In [ ]:
def show(dataset):
  loader = torch.utils.data.DataLoader(dataset, batch_size= 16, shuffle = True)
  batch = next(iter(loader))
  images, labels = batch

  grid = torchvision.utils.make_grid(images,nrow = 4)
  img = torchvision.transforms.ToPILImage()(grid)
  plt.imshow(img)

In [ ]:
show(Train_Ds)

In [ ]:
def set_device():
  if torch.cuda.is_available():
    dev = "cuda:0"
  else:
    dev = "cpu"
  device = torch.device(dev)
  return torch.device(dev)

In [ ]:
def tran_nn(model, train_loader, test_loader, criterion, optimizer, n_epochs):
  device = set_device()

  #to save best model
  best_acc = 0

  for epoch in range (n_epochs):
    print(epoch)
    model.train()
    running_loss = 0.0
    runnin_correct = 0.0
    total = 0
    for data in train_loader:
      images, labels = data
      images = images.to(device)
      labels = labels.to(device)
      total += labels.size(0)
      optimizer.zero_grad()

      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)
      loss = criterion(outputs, labels)

      #backpropegate
      loss.backward()
      optimizer.step()
      running_loss += loss.item()
      runnin_correct += (labels == predicted).sum().item()

    #print result values
    print("yes")
    epoch_loss = running_loss/len(train_loader)
    epoch_acc = 100.00 * runnin_correct / total
    print("Training dataset. ")
    print("correct prediction ", runnin_correct, "       out of ", total)
    print("epoch accuracy is  ", epoch_acc)
    print("epoch loss is ", epoch_loss)
    #print("    -Training dataset. Got %d out of %d images correctly ( %.3f%%). Epoch loss: %.03f") %  (runnin_correct, total, , epoch_acc, epoch_loss )
    test_data_acc = evaluate_model(model, test_loader)
    if (test_data_acc > best_acc):
      best_acc = test_data_acc
      save_checkpoint(model, epoch, optimizer, best_acc)
  print("finished")
  return model


In [ ]:
def save_checkpoint(model, epoch, optimizer, best_acc):
  state = {
      'epoch' : epoch + 1,
      'best_model' : model.state_dict(),
      'best accuracy' : best_acc,
      'optimizer' : optimizer.state_dict()
  }
  #model will save model
  #optimizer for resume training
  torch.save(state, './drive/MyDrive/bestModel.pth')

In [ ]:
def evaluate_model(model,test_loader):
  model.eval()
  predicted_correctly = 0
  total = 0
  device = set_device()

  #because we dont nee backpropegate while evaluating
  with torch.no_grad():
    for data in test_loader:
      images, labels = data
      images = images.to(device)
      labels = labels.to(device)
      total += labels.size(0)
      
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)
      predicted_correctly += (predicted == labels).sum().item()

  epoch_acc = 100.00 * predicted_correctly / total
  print("Test dataset. ")
  print("correct prediction ", predicted_correctly, "       out of " ,total)
  print("epoch accuracy is  ", epoch_acc)
  return epoch_acc
  



In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:


model=Net()



In [ ]:
number_features = model.fc.in_features
number_classes = 10 #depends upon dataset
model.fc = nn.Linear(number_features, number_classes)
device = set_device()
model = model.to(device)
loss_ftn = nn.CrossEntropyLoss() #loss ftn for classification problem
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.9, weight_decay = 0.003)


In [ ]:
tran_nn(model,train_loader, test_loader, loss_ftn, optimizer,2  )

In [ ]:
checkpoint = torch.load("./drive/MyDrive/bestModel.pth")

In [ ]:
print(checkpoint['epoch'])
print(checkpoint['best accuracy'])

In [ ]:
#to save model
model = models.resnet18()
number_features = model.fc.in_features
number_classes = 10 #depends upon dataset
model.fc = nn.Linear(number_features, number_classes)
model.load_state_dict(checkpoint['best_model'])
torch.save(model,"./drive/MyDrive/bestModel_final2.pth" )